In [ ]:
#Conda 환경 설정
!conda create -n convert
!conda activate convert
!conda install jupyter notebook
!python3 -m ipykernel install --user --name convert --display-name "convert"

In [ ]:
#update
!sudo apt update --yes
!sudo apt upgrade --yes
!sudo apt autoremove --yes

In [ ]:
#lib 설치
!conda install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch
!conda install -c conda-forge onnx

In [ ]:
%cd /home/ai/Desktop/work/ai_boostcamp/final/pth_convert

Pytorch를 ONNX로 변환

In [ ]:
import torch
import onnx
#https://github.com/pytorch/vision/blob/main/torchvision/models/segmentation/deeplabv3.py

#customized model
from models.deeplabv3 import deeplabv3_mobilenet_v3

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch_model = deeplabv3_mobilenet_v3(
    pretrained_backbone=False,
    aux_loss=False,
    small=True,
    reduced_tail=True,
    grid_mode=True,
).to(device)

model_path = './dlv3_mbv3-small_reduced_grid.211222.pth'
tmp = torch.load(model_path)
for w_name in list(tmp.keys())[-8:]:
    del tmp[w_name]
torch_model.load_state_dict(tmp)

torch_model.eval()

batch_size = 1
x = torch.randn(batch_size, 3, 240, 320).to(device)

torch.onnx.export(torch_model,
                  x,
                  "onnx.onnx",
                  export_params=True,
                  opset_version=10,
                  do_constant_folding=True,
                  verbose = True,
                  input_names = ['actual_input_1'],
                  output_names = ['output1']
                  )
onnx_model = onnx.load("onnx.onnx")
onnx.checker.check_model(onnx_model)
onnx.helper.printable_graph(onnx_model.graph)

ONNX를 TensorFlow로 변환

In [ ]:
!pip install onnx_tf
!sudo apt update --yes
!sudo apt install python3-dev python3-pip python3-venv --yes
!pip install --upgrade tensorflow

In [ ]:
from onnx_tf.backend import prepare
import onnx

TF_PATH = "./pb.pb"
ONNX_PATH = "./onnx.onnx"
onnx_model = onnx.load(ONNX_PATH)
tf_rep = prepare(onnx_model)
tf_rep.export_graph(TF_PATH)

TensorFlow를 TensorFlow Lite로 변환

In [ ]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model("./pb.pb")

#optimization
#https://github.com/sithu31296/PyTorch-ONNX-TFLite
converter.optimizations = [tf.lite.Optimize.DEFAULT]
#https://www.tensorflow.org/lite/performance/post_training_quantization?hl=ko
converter.target_spec.supported_types = [tf.float16]
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
tflite_model = converter.convert()

with open('tflite.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
import numpy as np
import tensorflow as tf
from PIL import Image 
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

filename = './test_images/MP_SEL_SUR_000004.jpg'

input_image = Image.open(filename)
input_image = input_image.convert("RGB")
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((240, 320)),
])
input_data = preprocess(input_image)
input_data = input_data.unsqueeze(0)

print(input_data.size())
interpreter = tf.lite.Interpreter(model_path='tflite.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

output_data = interpreter.get_tensor(output_details[0]['index'])
output_data = output_data.argmax(1)
input_data = input_data[0].cpu().numpy().transpose([1, 2, 0])
print(input_data.shape)
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

import matplotlib.pyplot as plt
fig, axes = plt.subplots(1, 2, dpi=200)
axes[0].imshow(output_data[0])
axes[1].imshow(input_data)